## Clone YOLOv9

In [1]:
# # clone YOLO v9
!git clone https://github.com/WongKinYiu/yolov9.git

GPU check

In [2]:
# !nvidia-smi

## Get pre-train model YOLOv9-e, gelan-c

In [3]:
# # get pretrained-model
!wget -P /kaggle/working/yolov9 https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e-converted.pt
!wget -P /kaggle/working/yolov9 https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt

In [4]:
%cd /kaggle/working/yolov9
!pip install -r requirements.txt

Change gelan config to 1 class

In [5]:
!cp /kaggle/working/yolov9/models/detect/gelan-c.yaml /kaggle/working/yolov9/models/detect/YOLOv9-gelan-c.yaml

In [6]:
%%writefile /kaggle/working/yolov9/data/hyps/hyp.scratch-high.yaml

lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 7.5  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
dfl: 1.5  # dfl loss gain
iou_t: 0.20  # IoU training threshold
anchor_t: 5.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.9  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.15  # image mixup (probability)
copy_paste: 0.3  # segment copy-paste (probability)


In [7]:
%%writefile /kaggle/working/yolov9/models/detect/YOLOv9-gelan-c.yaml

# YOLOv9

# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
#activation: nn.LeakyReLU(0.1)
#activation: nn.ReLU()

# anchors
anchors: 3

# gelan backbone
backbone:
  [
   # conv down
   [-1, 1, Conv, [64, 3, 2]],  # 0-P1/2

   # conv down
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4

   # elan-1 block
   [-1, 1, RepNCSPELAN4, [256, 128, 64, 1]],  # 2

   # avg-conv down
   [-1, 1, ADown, [256]],  # 3-P3/8

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 256, 128, 1]],  # 4

   # avg-conv down
   [-1, 1, ADown, [512]],  # 5-P4/16

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 6

   # avg-conv down
   [-1, 1, ADown, [512]],  # 7-P5/32

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 8
  ]

# gelan head
head:
  [
   # elan-spp block
   [-1, 1, SPPELAN, [512, 256]],  # 9

   # up-concat merge
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 12

   # up-concat merge
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [256, 256, 128, 1]],  # 15 (P3/8-small)

   # avg-conv-down merge
   [-1, 1, ADown, [256]],
   [[-1, 12], 1, Concat, [1]],  # cat head P4

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 18 (P4/16-medium)

   # avg-conv-down merge
   [-1, 1, ADown, [512]],
   [[-1, 9], 1, Concat, [1]],  # cat head P5

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 21 (P5/32-large)

   # detect
   [[15, 18, 21], 1, DDetect, [nc]],  # DDetect(P3, P4, P5)
  ]

In [8]:
# # %load /kaggle/working/yolov9/models/detect/YOLOv9-gelan-c.yaml

# # YOLOv9

# # parameters
# nc: 1  # number of classes
# depth_multiple: 1.0  # model depth multiple
# width_multiple: 1.0  # layer channel multiple
# #activation: nn.LeakyReLU(0.1)
# #activation: nn.ReLU()

# # anchors
# anchors: 3

# # gelan backbone
# backbone:
#   [
#    # conv down
#    [-1, 1, Conv, [64, 3, 2]],  # 0-P1/2

#    # conv down
#    [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4

#    # elan-1 block
#    [-1, 1, RepNCSPELAN4, [256, 128, 64, 1]],  # 2

#    # avg-conv down
#    [-1, 1, ADown, [256]],  # 3-P3/8

#    # elan-2 block
#    [-1, 1, RepNCSPELAN4, [512, 256, 128, 1]],  # 4

#    # avg-conv down
#    [-1, 1, ADown, [512]],  # 5-P4/16

#    # elan-2 block
#    [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 6

#    # avg-conv down
#    [-1, 1, ADown, [512]],  # 7-P5/32

#    # elan-2 block
#    [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 8
#   ]

# # gelan head
# head:
#   [
#    # elan-spp block
#    [-1, 1, SPPELAN, [512, 256]],  # 9

#    # up-concat merge
#    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
#    [[-1, 6], 1, Concat, [1]],  # cat backbone P4

#    # elan-2 block
#    [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 12

#    # up-concat merge
#    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
#    [[-1, 4], 1, Concat, [1]],  # cat backbone P3

#    # elan-2 block
#    [-1, 1, RepNCSPELAN4, [256, 256, 128, 1]],  # 15 (P3/8-small)

#    # avg-conv-down merge
#    [-1, 1, ADown, [256]],
#    [[-1, 12], 1, Concat, [1]],  # cat head P4

#    # elan-2 block
#    [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 18 (P4/16-medium)

#    # avg-conv-down merge
#    [-1, 1, ADown, [512]],
#    [[-1, 9], 1, Concat, [1]],  # cat head P5

#    # elan-2 block
#    [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 21 (P5/32-large)

#    # detect
#    [[15, 18, 21], 1, DDetect, [nc]],  # DDetect(P3, P4, P5)
#   ]


## Train with custom dataset - Gelan-c

In [9]:
### Train model with custome data

%cd /kaggle/working/yolov9 

data_yaml_path = "/kaggle/input/braintumor/data.yaml"
cfg_path = "/kaggle/working/yolov9/models/detect/YOLOv9-gelan-c.yaml"
weight_path = "/kaggle/working/yolov9/gelan-c.pt"
hyps_path = "/kaggle/working/yolov9/data/hyps/hyp.scratch-high.yaml"

!python train.py --workers 8 --device 0 --batch 16 --data {data_yaml_path} --img 640 --cfg {cfg_path} --weights {weight_path} --name gelan-c-ver --hyp {hyps_path} --min-items 0 --epochs 25 --close-mosaic 15  #--freeze 10

# train gelan models
# python train.py --workers 8 --device 0 --batch 32 --data data/coco.yaml --img 640 --cfg models/detect/gelan-c.yaml --weights '' --name gelan-c --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15

## Train with custom dataset - YOLOv9

In [10]:
!cp /kaggle/working/yolov9/models/detect/yolov9-c.yaml /kaggle/working/yolov9/models/detect/YOLOv9-yolov9-c.yaml

In [11]:
%%writefile /kaggle/working/yolov9/models/detect/YOLOv9-yolov9-c.yaml
# YOLOv9

# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
#activation: nn.LeakyReLU(0.1)
#activation: nn.ReLU()

# anchors
anchors: 3

# YOLOv9 backbone
backbone:
  [
   [-1, 1, Silence, []],  
   
   # conv down
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2

   # conv down
   [-1, 1, Conv, [128, 3, 2]],  # 2-P2/4

   # elan-1 block
   [-1, 1, RepNCSPELAN4, [256, 128, 64, 1]],  # 3

   # avg-conv down
   [-1, 1, ADown, [256]],  # 4-P3/8

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 256, 128, 1]],  # 5

   # avg-conv down
   [-1, 1, ADown, [512]],  # 6-P4/16

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 7

   # avg-conv down
   [-1, 1, ADown, [512]],  # 8-P5/32

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 9
  ]

# YOLOv9 head
head:
  [
   # elan-spp block
   [-1, 1, SPPELAN, [512, 256]],  # 10

   # up-concat merge
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 7], 1, Concat, [1]],  # cat backbone P4

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 13

   # up-concat merge
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 5], 1, Concat, [1]],  # cat backbone P3

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [256, 256, 128, 1]],  # 16 (P3/8-small)

   # avg-conv-down merge
   [-1, 1, ADown, [256]],
   [[-1, 13], 1, Concat, [1]],  # cat head P4

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 19 (P4/16-medium)

   # avg-conv-down merge
   [-1, 1, ADown, [512]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 22 (P5/32-large)
   
   
   # multi-level reversible auxiliary branch
   
   # routing
   [5, 1, CBLinear, [[256]]], # 23
   [7, 1, CBLinear, [[256, 512]]], # 24
   [9, 1, CBLinear, [[256, 512, 512]]], # 25
   
   # conv down
   [0, 1, Conv, [64, 3, 2]],  # 26-P1/2

   # conv down
   [-1, 1, Conv, [128, 3, 2]],  # 27-P2/4

   # elan-1 block
   [-1, 1, RepNCSPELAN4, [256, 128, 64, 1]],  # 28

   # avg-conv down fuse
   [-1, 1, ADown, [256]],  # 29-P3/8
   [[23, 24, 25, -1], 1, CBFuse, [[0, 0, 0]]], # 30  

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 256, 128, 1]],  # 31

   # avg-conv down fuse
   [-1, 1, ADown, [512]],  # 32-P4/16
   [[24, 25, -1], 1, CBFuse, [[1, 1]]], # 33 

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 34

   # avg-conv down fuse
   [-1, 1, ADown, [512]],  # 35-P5/32
   [[25, -1], 1, CBFuse, [[2]]], # 36

   # elan-2 block
   [-1, 1, RepNCSPELAN4, [512, 512, 256, 1]],  # 37
   
   
   
   # detection head

   # detect
   [[31, 34, 37, 16, 19, 22], 1, DualDDetect, [nc]],  # DualDDetect(A3, A4, A5, P3, P4, P5)
  ]


In [12]:
### Train model with custome data

%cd /kaggle/working/yolov9 

data_yaml_path = "/kaggle/input/braintumor/data.yaml"
cfg_path = "/kaggle/working/yolov9/models/detect/YOLOv9-yolov9-c.yaml"
weight_path = "/kaggle/working/yolov9/yolov9-e-converted.pt"
hyps_path = "/kaggle/working/yolov9/data/hyps/hyp.scratch-high.yaml"

!python train_dual.py --workers 8 --device 0 --batch 8 --data {data_yaml_path} --img 640 --cfg {cfg_path} --weights {weight_path} --name yolov9-c-ver --hyp {hyps_path} --min-items 0 --epochs 15 --close-mosaic 15  #--freeze 10

# train gelan models
# python train.py --workers 8 --device 0 --batch 32 --data data/coco.yaml --img 640 --cfg models/detect/gelan-c.yaml --weights '' --name gelan-c --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15

## Train with custom dataset - YOLOv9 freeze 5

In [14]:
### Train model with custome data

%cd /kaggle/working/yolov9 

data_yaml_path = "/kaggle/input/braintumor/data.yaml"
cfg_path = "/kaggle/working/yolov9/models/detect/YOLOv9-yolov9-c.yaml"
weight_path = "/kaggle/working/yolov9/yolov9-e-converted.pt"
hyps_path = "/kaggle/working/yolov9/data/hyps/hyp.scratch-high.yaml"

!python train_dual.py --workers 8 --device 0 --batch 16 --data {data_yaml_path} --img 640 --cfg {cfg_path} --weights {weight_path} --name yolov9-c-freeze --hyp {hyps_path} --min-items 0 --epochs 35 --close-mosaic 15  --freeze 5

# train gelan models
# python train.py --workers 8 --device 0 --batch 32 --data data/coco.yaml --img 640 --cfg models/detect/gelan-c.yaml --weights '' --name gelan-c --hyp hyp.scratch-high.yaml --min-items 0 --epochs 500 --close-mosaic 15

[Errno 2] No such file or directory: '/kaggle/working/yolov9'
/kaggle/working
python: can't open file '/kaggle/working/train_dual.py': [Errno 2] No such file or directory
